# Task 4.2 - Autoencoder

Next, a UNet based magnetic resonance image segementation of the brain is used. The segmentation accuracy of the model will need to be validated and achieve > 0.9 DSC for all labels. Need to use categorical (one-hot) output in the network and must also visualise some of the segmentation results to justify the DSC (F1) scores obtrained. 

My understanding of U-net is as follows - it is a CNN used for specifically biomedical segmentation. There are two key symmetrical parts - the *encoder* and the *decoder*. What makes a U-net unique is that it includes skip connections from the encoder-decoder symmetr, uses fully convolutional design only with no dense layers, and has a U-shaped design that is fully symmetrical.

Our 'X' input now is the original input, and the 'segmented' images are our 'classification' in the other folder. From the segmented images we can see (with the one hot encoding) that there are 4 key categories each value can fit in:
1. Black (presumably background)
2. Dark Grey (potenitally 'CSF' when looking at online images)
3. Grey (potenitally grey matter when looking at online images)
4. Light Grey (potentially white matter when looking at online images)

Therefore we need to find a way to have the UNET be able to classify each pixel into one of these four categories.

In [ ]:
# Range of Imports
import torch
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
from mpl_toolkits.axes_grid1 import ImageGrid
from torchvision.utils import save_image, make_grid
from torch.optim import Adam

In [ ]:
# Our 